# บดทดลองดืงข้มุนจาก github เลื่อง Covid-19 on 11-04-2020
* data source:
* h 


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(f'pandas version:{pd.__version__}')
print(f'seaborn version:{sns.__version__}')

In [ ]:
pd.Timestamp.now()

## read data from github

In [ ]:
df=pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df


In [ ]:
path='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
#path='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
status=['Confirmed', 'Deaths', 'Recovered']
dfs=[] # list contains serveral dataframes
for s in status:
#     print(f'{path}{s.lower()}_global.csv')
    d=pd.read_csv(f'{path}{s.lower()}_global.csv')
    dfs.append(d)
dfs


In [ ]:
dfs[0]

In [ ]:
dfs[1]

In [ ]:
dfs[2]

### appen data

In [ ]:
df=pd.concat(dfs, keys=status)
df

In [ ]:
df.loc['Deaths']

In [ ]:
df.loc['Confirmed']

In [ ]:
df.loc['Recovered']

In [ ]:
# df.drop(columns=['Lat','Long', inplace=True])

### unpivot/melt data

In [ ]:
pd.melt(df.loc['Recovered'], 
        id_vars=['Province/State','Country/Region','Lat','Long'],
        var_name='Date',value_name='Confirmed')

In [ ]:
dtmps=[]
for k in status:
    dtmps.append(pd.melt(df.loc[k], 
          id_vars= ['Province/State','Country/Region','Lat','Long'],
          var_name='Date',value_name= k))

In [ ]:
dtmps

In [ ]:
dtmps[0]

In [ ]:
dtmps[1]

In [ ]:
dtmps[2]

In [ ]:
dx=[d.set_index(['Province/State','Country/Region','Lat','Long','Date']) for d in dtmps]
dcovid=pd.concat(dx, axis=1)
dcovid.reset_index(inplace=True)


In [ ]:
dcovid

In [ ]:
dcovid.columns

### remove duplicated columns

In [ ]:
dcovid.columns

In [ ]:
dcovid.columns.duplicated()

In [ ]:
dcovid=dcovid.loc[:,~dcovid.columns.duplicated()]
dcovid

### Convert date data type

In [ ]:
dcovid.info()

In [ ]:
dcovid['Date']=pd.to_datetime(dcovid['Date'])
# dcovid.loc[:, 'Date']=pd.to_datetime(dcovid['Date'])

In [ ]:
dcovid.info()

In [ ]:
dcovid.head()

In [ ]:
dcovid=dcovid.sort_values(['Country/Region','Province/State','Date']).reset_index(drop=True)
dcovid

In [ ]:
pd.set_option('display.max_rows',100)

In [ ]:
dcovid[dcovid['Country/Region']=='Laos']

### aggregate by Country/Region

In [ ]:
dcovid[dcovid['Country/Region']=='China']

In [ ]:
dcountry=dcovid.groupby(['Country/Region', 'Date']).sum()[['Confirmed', 'Deaths', 'Recovered']]
dcountry

In [ ]:
dcountry.index

In [ ]:
dcountry.loc['Laos']

In [ ]:
dcountry.loc['Laos', '2020-04-04']

#### ເພື່ອສະດວກໃນການສ້າງເສັ້ນສະແດງ(grap)ຄວນລີເຊັດອີນເຕັກອອກ(reset_index)

In [ ]:
dcountry.reset_index(inplace=True)
dcountry

In [ ]:
dcountry.index

### ອະທີບາຍແຖວທີ່ມີ diff (ແມ່ນເອົາວັກແຖວລຸ່ມມາລົບໃຫ້ວັກແຖວເທີງເພີ່ືອສະແດງໃຫ້ເຫັນການເພີ່ມຂື້ນແຕ່ລະວັນ)
* ວັກທີ່ມີຄຳວ່າ pct_change ແມ່ນສະແດງເປັນເບີເຊັນ %

In [ ]:
dcountry['daily_new_patient']=dcountry.groupby(['Country/Region'])['Confirmed'].diff()
dcountry['daily_new_patient_pct']=dcountry.groupby(['Country/Region'])['Confirmed'].pct_change()

In [ ]:
dcountry[dcountry['Country/Region']=='Italy']

In [ ]:
dcountry[dcountry['Country/Region']=='Laos']

### interactive chart

### All Countries

In [ ]:
pd.Series(dcountry['Country/Region'].unique()).sort_values()

In [ ]:
@interact(country=pd.Series(dcountry['Country/Region'].unique()).sort_values())
def plot_chart(country):
    df=dcountry[dcountry['Country/Region'].str.contains(country)]
    df[['Date', 'Confirmed', ]].set_index('Date').plot(figsize=(16, 7))

### countries with confirmed cases > n

In [ ]:
dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 300
dq.index[dq.Confirmed]

@interact(country=pd.Series(dq.index[dq.Confirmed]))
def plot_chart(country):
    df=dcountry[dcountry['Country/Region'].str.contains(country)]
    df[['Date', 'Confirmed', 'Recovered', 'Deaths']].set_index('Date').plot(figsize=(16, 7), 
                                                                            color=['purple', 'green', 'red'], 
                                                                            drawstyle='steps')
    plt.xlabel('')
    plt.ylabel('persons')
    plt.title(f'COVID-19 in {country}')

### daily new confirmed cases

In [ ]:
dcountry[dcountry['Country/Region']=='Laos']

In [ ]:
plt.figure(figsize=(16,6))
sns.barplot(data=dcountry[dcountry['Country/Region']=='Laos'], 
        x='Date',y='daily_new_patient', color='.7')

In [ ]:
plt.figure(figsize=(16, 6))
dc=dcountry[dcountry['Country/Region']=='Thailand'].set_index('Date')
dc['daily_new_patient'].plot(kind='bar', color='.7')

## Log scale

In [ ]:
country1='Malaysia'
country2='Thailand'
fig, ax = plt.subplots(figsize=(10, 7))
ax.set_yscale('log', basey=10)
dc1=dcountry[dcountry['Country/Region']==country1]
dc2=dcountry[dcountry['Country/Region']==country2]
sns.lineplot(x=dc1.Date, y=dc1['Confirmed'], ax=ax, color='salmon', label=country1)
sns.lineplot(x=dc2.Date, y=dc2['Confirmed'], ax=ax, color='lightblue', label=country2)
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
dc=dcountry[dcountry['Country/Region'].str.contains('US')]
sns.lineplot(x=dc.Date, y=dc['Confirmed'], ax=ax, color='salmon', label='standard')
ax2=ax.twinx()
ax2.set_yscale('log', basey=10)
sns.lineplot(x=dc.Date, y=dc['Confirmed'], ax=ax2, color='black', label='log')
plt.title('COVID-19 in Thailand')
plt.grid()
fig.legend()

In [ ]:
dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 10000
dq.index[dq.Confirmed]
dg=dcountry[dcountry['Country/Region'].isin(dq.index[dq.Confirmed])].reset_index(drop=True)
dg

In [ ]:
sns.catplot(data=dg, x='Date', y='Confirmed', kind='point', col='Country/Region', col_wrap=4, )